In [1]:
import Pkg; Pkg.activate(".")
using Clapeyron
includet("./saftvrmienn.jl")
# These are functions we're going to overload for SAFTVRMieNN
import Clapeyron: a_res, saturation_pressure, pressure

using Flux
using Plots, Statistics
using ForwardDiff, DiffResults

using Zygote, ChainRulesCore
using ImplicitDifferentiation

using CSV, DataFrames
using MLUtils
using RDKitMinimalLib
using JLD2

# Multithreaded loss
using Zygote: bufferfrom
using Base.Threads: @spawn

  Activating project at `~/SAFT_ML`


In [6]:
function pressure_NN(model, V, T)
    return ForwardDiff.derivative(V -> eos(model, V, T), V)
end

function volume_NN(X, p, T)
    model = make_model(X...)
    V = volume(model, p, T; phase=:liquid)

    return V
end

#! In the forward pass, use ForwardDiff to compute value + gradient
#! Then return cached value in pullback function
function ChainRulesCore.rrule(::typeof(volume_NN), X, p, T)
    model = make_model(X...)
    # p, Vₗ, Vᵥ = saturation_pressure(model, T)
    vL = volume(model, p, T, [1.0]; phase=:liquid)

    function f_pullback(Δy)
        #* Newton step from perfect initialisation
        function f_V(X, p, T)
            model = make_NN_model(X...)
            ∂p∂V = ForwardDiff.derivative(V -> pressure_NN(model, V, T), vL)
            v2 = vL - (pressure_NN(model, vL, T) - p) / ∂p∂V
            return v2
        end

        ∂X = @thunk(ForwardDiff.gradient(X -> f_V(X, p, T), X) .* Δy)
        ∂p = @thunk(ForwardDiff.derivative(p -> f_V(X, p, T), p) .* Δy)
        ∂T = @thunk(ForwardDiff.derivative(T -> f_V(X, p, T), T) .* Δy)
        return (NoTangent(), ∂X, ∂p, ∂T)
    end

    return p, f_pullback
end

X = [16.04, 1.0, 3.737, 6.0, 12.504, 152.58]
V = volume_NN(X, 1e7, 100.0)
∂V∂X = Zygote.gradient(X -> volume_NN(X, 1e7, 100.0), X)

([-0.0, 2.260005096581295e-5, 2.1863843008154135e-5, 1.46022174535494e-6, 1.6493904176981177e-7, -6.011351535625539e-8],)

In [ ]:
# Differentiate through volume solver

function saturation_pressure_NN(X, T)
    model = make_model(X...)
    p, Vₗ, Vᵥ = saturation_pressure(model, T)

    return p
end

#! In the forward pass, use ForwardDiff to compute value + gradient
#! Then return cached value in pullback function
function ChainRulesCore.rrule(::typeof(saturation_pressure_NN), X, T)
    model = make_model(X...)
    p, Vₗ, Vᵥ = saturation_pressure(model, T)

    function f_pullback(Δy)
        #* Newton step from perfect initialisation
        function f_p(X, T)
            model = make_NN_model(X...)
            p2 = -(eos(model, Vᵥ, T) - eos(model, Vₗ, T)) / (Vᵥ - Vₗ)
            return p2
        end

        ∂X = @thunk(ForwardDiff.gradient(X -> f_p(X, T), X) .* Δy)
        ∂T = @thunk(ForwardDiff.derivative(T -> f_p(X, T), T) .* Δy)
        return (NoTangent(), ∂X, ∂T)
    end

    return p, f_pullback
end